In [45]:
import os 
import numpy as np
import pandas as pd 
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.api as sm
from statsmodels.formula.api import ols
warnings.filterwarnings("ignore")
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from patsy import dmatrices
from sklearn.metrics import r2_score 
import math

In [2]:
# # change directories - run only once
# os.chdir(os.path.dirname(os.getcwd()))
# os.chdir(os.path.dirname(os.getcwd()))
# os.chdir('Data')

In [80]:
# read in data
df = pd.read_csv('od_full.csv', dtype = {'FIPS': str})

# define RMSE function
def compute_rmse(predictions, yvalues): 
    rmse = np.sqrt(np.sum(np.subtract(yvalues, predictions)**2)/len(yvalues))
    return rmse 

In [82]:
# fit model with just spatial mean component
mod = ols('Overdose_Rate_per_100k ~ Spatial_Mean', data = df)
res = mod.fit()
print(res.summary())

                              OLS Regression Results                              
Dep. Variable:     Overdose_Rate_per_100k   R-squared:                       0.490
Model:                                OLS   Adj. R-squared:                  0.490
Method:                     Least Squares   F-statistic:                     8247.
Date:                    Thu, 28 Apr 2022   Prob (F-statistic):               0.00
Time:                            16:20:36   Log-Likelihood:                -32881.
No. Observations:                    8600   AIC:                         6.577e+04
Df Residuals:                        8598   BIC:                         6.578e+04
Df Model:                               1                                         
Covariance Type:                nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercep

In [83]:
# estimating the missing overdose rate values with our model
# this line of code needs to be changed when we add more predictors
od_pred = res.predict({'Spatial_Mean': df.loc[np.where(df['Overdose_Rate_per_100k'].isnull())[0]]['Spatial_Mean']})

# a new df with missing values filled from our prediction
df2 = df
df2['Overdose_Rate_per_100k'][od_pred.index] = od_pred

In [79]:
df2.to_csv('test_estimate.csv', index = False)

In [50]:
df.loc[np.where(df['Overdose_Rate_per_100k'].isnull())[0]]

,Year,FIPS,County,State,Region,Division,Overdose_Deaths,Overdose_Pop,Overdose_Rate_per_100k,Population,...,Pct_Unemployed,Pct_Poverty,Median_HHI,Jail_Pop,Urbanicity,Spatial_Max,Spatial_Mean,Prev_Spatial_Max,Prev_Spatial_Mean,Geometry
0,2011,1001,"Autauga County, AL",Alabama,South,East South Central,NaN,NaN,NaN,50756.0,...,8.3,14.9,48863.0,161.00,2.0,NaN,NaN,25.204500,16.526161,"POLYGON ((-86.496774 32.344437, -86.717897 32...."
2,2011,1005,"Barbour County, AL",Alabama,South,East South Central,NaN,NaN,NaN,29737.0,...,11.4,29.5,30117.0,105.00,1.0,NaN,NaN,NaN,NaN,"POLYGON ((-85.056029 32.063055, -85.050212 32...."
3,2011,1007,"Bibb County, AL",Alabama,South,East South Central,NaN,NaN,NaN,21587.0,...,10.4,22.2,37347.0,67.50,3.0,13.640773,12.565340,25.204500,15.378684,"POLYGON ((-87.4212 32.874508, -87.420132 32.90..."
4,2011,1009,"Blount County, AL",Alabama,South,East South Central,NaN,NaN,NaN,58345.0,...,8.6,14.9,41940.0,127.50,3.0,55.504718,23.305981,50.728854,22.318995,"POLYGON ((-86.577799 33.765316, -86.759144 33...."
5,2011,1011,"Bullock County, AL",Alabama,South,East South Central,NaN,NaN,NaN,10985.0,...,11.5,32.8,26038.0,32.57,1.0,NaN,NaN,7.847822,7.847822,"POLYGON ((-85.410241 32.146651, -85.428476 32...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31414,2020,56035,"Sublette County, WY",Wyoming,West,Mountain,NaN,NaN,NaN,9813.0,...,7.2,6.2,82442.0,16.00,1.0,35.151032,35.151032,28.017626,28.017626,"POLYGON ((-110.05157 43.464783, -109.754717 43..."
31416,2020,56039,"Teton County, WY",Wyoming,West,Mountain,NaN,NaN,NaN,23081.0,...,6.0,5.2,92488.0,25.00,1.0,18.013002,18.013002,28.017626,20.953692,"POLYGON ((-111.048751 44.060403, -111.048751 4..."
31417,2020,56041,"Uinta County, WY",Wyoming,West,Mountain,NaN,NaN,NaN,20299.0,...,6.3,8.5,71246.0,55.00,1.0,35.151032,35.151032,NaN,NaN,"POLYGON ((-111.046402 41.579845, -110.062787 4..."
31418,2020,56043,"Washakie County, WY",Wyoming,West,Mountain,NaN,NaN,NaN,7885.0,...,5.3,9.7,58532.0,28.00,1.0,19.798305,19.798305,28.017626,22.148260,"POLYGON ((-108.550758 44.079403, -108.550562 4..."
